# ETL Processes
This notebook to develop the ETL process for each table before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries_1 import *

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=trtgv")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    return all_files

The function get_files takes a filepath parameter, which represents the directory path where the files are located.

The variable all_files is initialized as an empty list. This list will store the absolute paths of all the JSON files found in the directory and its subdirectories.

The os.walk function is used to iterate over all the directories and files within the filepath directory. It returns a generator that yields a 3-tuple (root, dirs, files) for each directory it encounters.

Within each iteration of os.walk, the variable files is assigned the list of files found in the current directory. The glob.glob function is used with the pattern os.path.join(root, '*.json') to retrieve only the JSON files in the directory. os.path.join is used to construct the absolute path of each file by joining the root directory and the file name with a path separator.

For each file f in the files list, the absolute path of the file is appended to the all_files list using os.path.abspath(f).

After iterating through all the directories and files in the filepath directory, the function returns the all_files list, which contains the absolute paths of all the JSON files found.

This code essentially walks through the directory structure starting from filepath, finds all the JSON files, and collects their absolute paths in the all_files list.

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data')

In [13]:
filepath = song_files[0]  # first file

In [14]:
filepath

'C:\\Users\\Adel\\Project01-Data-Modeling-Postgres\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json'

In [16]:
df = pd.read_json(filepath, lines = True)
df.head(5)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [17]:
df.values[0].tolist()

[1,
 'ARD7TVE1187B99BFB1',
 nan,
 nan,
 'California - LA',
 'Casual',
 'SOMZWCG12A8C13C480',
 "I Didn't Mean To",
 218.93179,
 0]

In [18]:
values = {}
for column in df.columns:
    values[column] = df[column].values[0]
print(values)

{'num_songs': 1, 'artist_id': 'ARD7TVE1187B99BFB1', 'artist_latitude': nan, 'artist_longitude': nan, 'artist_location': 'California - LA', 'artist_name': 'Casual', 'song_id': 'SOMZWCG12A8C13C480', 'title': "I Didn't Mean To", 'duration': 218.93179, 'year': 0}


In [19]:
for key,v in values.items():
    print(v, type(v))

1 <class 'numpy.int64'>
ARD7TVE1187B99BFB1 <class 'str'>
nan <class 'numpy.float64'>
nan <class 'numpy.float64'>
California - LA <class 'str'>
Casual <class 'str'>
SOMZWCG12A8C13C480 <class 'str'>
I Didn't Mean To <class 'str'>
218.93179 <class 'numpy.float64'>
0 <class 'numpy.int64'>


*****************************

Song Table: song ID, title, artist ID, year, and duration

In [20]:
song_data = [values['song_id'], values['title'], values['artist_id'], int(values['year']), float(values['duration'])]
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [21]:
cur.execute(song_table_insert, song_data)
conn.commit()

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [22]:
values

{'num_songs': 1,
 'artist_id': 'ARD7TVE1187B99BFB1',
 'artist_latitude': nan,
 'artist_longitude': nan,
 'artist_location': 'California - LA',
 'artist_name': 'Casual',
 'song_id': 'SOMZWCG12A8C13C480',
 'title': "I Didn't Mean To",
 'duration': 218.93179,
 'year': 0}

In [23]:
artist_data = [values['artist_id'], values['artist_name'], values['artist_location'], float(values['artist_longitude']), float(values['artist_latitude'])]
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [28]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [29]:
log_files = get_files('data/log_data')

In [30]:
filepath = log_files[0]
filepath

'C:\\Users\\Adel\\Project01-Data-Modeling-Postgres\\data\\log_data\\2018\\11\\2018-11-01-events.json'

In [31]:
df = pd.read_json(filepath, lines = True)
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         11 non-null     object 
 1   auth           15 non-null     object 
 2   firstName      15 non-null     object 
 3   gender         15 non-null     object 
 4   itemInSession  15 non-null     int64  
 5   lastName       15 non-null     object 
 6   length         11 non-null     float64
 7   level          15 non-null     object 
 8   location       15 non-null     object 
 9   method         15 non-null     object 
 10  page           15 non-null     object 
 11  registration   15 non-null     int64  
 12  sessionId      15 non-null     int64  
 13  song           11 non-null     object 
 14  status         15 non-null     int64  
 15  ts             15 non-null     int64  
 16  userAgent      15 non-null     object 
 17  userId         15 non-null     int64  
dtypes: float64(1

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [32]:
df.shape

(15, 18)

In [33]:
df = df[df['page']=='NextSong']
df.shape

(11, 18)

In [34]:
t = pd.to_datetime(df['ts'], unit='ms') 
t.head()

2   2018-11-01 21:01:46.796
4   2018-11-01 21:05:52.796
5   2018-11-01 21:08:16.796
6   2018-11-01 21:11:13.796
7   2018-11-01 21:17:33.796
Name: ts, dtype: datetime64[ns]

In [35]:
time_data = []
for line in t:
    time_data.append([line, 
                      line.hour, 
                      line.day, 
                      line.week, 
                      line.month, 
                      line.year, 
                      line.day_name()])
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [36]:
time_df = pd.DataFrame.from_records(time_data, columns=column_labels)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,Thursday
1,2018-11-01 21:05:52.796,21,1,44,11,2018,Thursday
2,2018-11-01 21:08:16.796,21,1,44,11,2018,Thursday
3,2018-11-01 21:11:13.796,21,1,44,11,2018,Thursday
4,2018-11-01 21:17:33.796,21,1,44,11,2018,Thursday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [37]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row)
    conn.commit()
    #print(i,list(row))

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [38]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [39]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [43]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [44]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record, My Fact Table
    songplay_data = ( pd.to_datetime(row.ts, unit='ms'), 
                     int(row.userId), 
                     row.level, 
                     songid, 
                     artistid, 
                     row.sessionId, 
                     row.location, 
                     row.userAgent)
    
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

# Close Connection to Sparkify Database

In [45]:
conn.close()

# Implement `etl.py`
Use what We've completed in this notebook to implement `etl.py`.

# Testing the ON CONFLICT on the user table

In [46]:
user_df.values[0]

array([8, 'Kaylee', 'Summers', 'F', 'free'], dtype=object)

In [47]:
columns = ['userId', 'firstName', 'lastName', 'gender', 'level']
foo = pd.DataFrame([['69', 'Anabelle', 'Simpson', 'F', 'free'],['69', 'Anabelle', 'Simpson', 'F', 'paid']], columns = columns)
foo.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,paid


In [48]:
for i, row in foo.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

InterfaceError: cursor already closed